In [131]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np


df = pd.read_csv("EURUSD_Candlestick_1_Hour_BID_04.05.2003-15.04.2023.csv")
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)


In [132]:
df=df[0:5000]
backcandles = 15


def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, 2 if pivot low, 3 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0
    
    pivotHigh = 1
    pivotLow = 2
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle].low > df.iloc[i].low:
            pivotLow=0
        if df.iloc[candle].high < df.iloc[i].high:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

In [133]:
window=5
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window), axis=1)

In [134]:
def pointpos(x):
    if x['isPivot']==2:
        return x['low']-1e-3
    elif x['isPivot']==1:
        return x['high']+1e-3
    else:
        return np.nan
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

dfpl = df[300:450]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")

fig.show()

In [195]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Chargement des données
df = pd.read_csv("EURUSD_Candlestick_1_Hour_BID_04.05.2003-15.04.2023.csv")
df = df[df['volume'] != 0].reset_index(drop=True)
df = df.head(5000)

def detect_structure_points(df, window):
    df['isPivotHigh'] = False
    df['isPivotLow'] = False
    last_pivot_type = None
    last_pivot_index = 0

    for i in range(window, len(df) - window):
        current_high = df.iloc[i]['high']
        current_low = df.iloc[i]['low']
        max_high = max(df.iloc[i-window:i+window]['high'])
        min_low = min(df.iloc[i-window:i+window]['low'])

        # Vérifier si c'est un point haut
        if current_high == max_high and last_pivot_type != 'high':
            df.loc[i, 'isPivotHigh'] = True
            last_pivot_type = 'high'
            last_pivot_index = i

        # Vérifier si c'est un point bas
        elif current_low == min_low and last_pivot_type != 'low':
            df.loc[i, 'isPivotLow'] = True
            last_pivot_type = 'low'
            last_pivot_index = i

    return df

# Détection des points de structure
window = 3
df = detect_structure_points(df, window)

# Création des points de position
df['pointpos'] = np.nan
df.loc[df['isPivotHigh'], 'pointpos'] = df['high'] + 1e-3
df.loc[df['isPivotLow'], 'pointpos'] = df['low'] - 1e-3

# Afficher les bougies et les points pivots
dfpl = df[300:450]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

# Ajouter les points de pivot au graphique
fig.add_trace(go.Scatter(
    x=dfpl[dfpl['isPivotHigh']].index,
    y=dfpl[dfpl['isPivotHigh']]['pointpos'],
    mode='markers',
    marker=dict(size=10, color='red', symbol='circle'),
    name='Pivot High'
))

fig.add_trace(go.Scatter(
    x=dfpl[dfpl['isPivotLow']].index,
    y=dfpl[dfpl['isPivotLow']]['pointpos'],
    mode='markers',
    marker=dict(size=10, color='green', symbol='circle'),
    name='Pivot Low'
))

fig.show()

# Affichage du nombre de points de structure identifiés
print(f"Nombre de points hauts de structure : {df['isPivotHigh'].sum()}")
print(f"Nombre de points bas de structure : {df['isPivotLow'].sum()}")

Nombre de points hauts de structure : 446
Nombre de points bas de structure : 447


NameError: name 'current_high' is not defined

In [162]:
print(df['isPivot'] < 0) 


0       False
1       False
2       False
3       False
4       False
        ...  
4995    False
4996    False
4997    False
4998    False
4999    False
Name: isPivot, Length: 5000, dtype: bool
